# Libraries

In [1]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from utils.Validator import *
from utils.Combinations import *
from utils.Network import *
pd.set_option('display.max_columns', None)  
sns.set(rc={'figure.figsize':(11.7,8.27)})

# Data

In [13]:
df_01 = pd.read_csv(r"../data/interim/TeamMemberships.csv")
df_02 = pd.read_csv(r"../data/interim/Teams.csv")
df_03 = pd.read_csv(r"../data/interim/CompetitionTags.csv")
df_04 = pd.read_csv(r"../data/interim/Tags.csv")

In [14]:
df_01.head()

,TeamId,UserId
0,518,635
1,526,839
2,527,816
3,528,778
4,529,747


In [15]:
df_02.head()

,TeamId,CompetitionId,TeamLeaderId,TeamName,Medal,PrivateLeaderboardRank
0,496,2435,647.0,team1,NaN,83.0
1,497,2435,619.0,jonp,3.0,25.0
2,499,2435,663.0,Bwaas,NaN,100.0
3,500,2435,673.0,Thylacoleo,3.0,23.0
4,501,2435,435.0,pjonesdotcda,NaN,80.0


# Cruce para tener la competencia que pertenece el usuario
#### La tabla Teams.csv solo se tiene esta relacion para el lider, la logica es ponerle la competencia a todos los usarios segun la competencia que tiene el lider en el grupo.

In [16]:
df_cruce1 = pd.merge(df_02[["TeamId","CompetitionId","TeamLeaderId"]], df_01[["TeamId","UserId"]],  how='inner', left_on=['TeamId','TeamLeaderId'], right_on = ['TeamId','UserId'])

In [19]:
print('cantidad de lideres por team:',df_02.groupby('TeamId').TeamId.count().max())
print('cantidad de teams:',df_02.shape[0])

cantidad de lideres por team: 1
cantidad de teams: 5357344


In [20]:
print('cantidad de lideres por team:',df_cruce1.groupby('TeamId').TeamId.count().max())
print('cantidad de teams:',df_cruce1.shape[0])

cantidad de lideres por team: 1
cantidad de teams: 5327674


In [21]:
print('cantidad de teams que no cruzaron',df_02.shape[0]-df_cruce1.shape[0])

cantidad de teams que no cruzaron 29670


# Cruce para tener la etiqueta de los tag que tiene asociado una competencia
#### La tabla CompetitionTags.csv solo tiene el identificador del tag, pero no el descriptivo de ese codigo

In [26]:
df_cruce2 = pd.merge(df_03[["CompetitionId","TagId"]], df_04[["TagId","Name"]],  how='inner', left_on=['TagId'], right_on = ['TagId'])
df_cruce2.head()

,CompetitionId,TagId,Name
0,3486,14102,image data
1,3947,14102,image data
2,3978,14102,image data
3,4104,14102,image data
4,4406,14102,image data


In [27]:
print('cantidad de lideres por team:',df_03.groupby('CompetitionId').CompetitionId.count().max())
print('cantidad de teams:',df_03.shape[0])

cantidad de lideres por team: 5
cantidad de teams: 686


In [29]:
print('cantidad de lideres por team:',df_04.groupby('TagId').TagId.count().max())
print('cantidad de teams:',df_04.shape[0])

cantidad de lideres por team: 1
cantidad de teams: 320


In [30]:
print('cantidad de lideres por team:',df_cruce2.groupby('CompetitionId').CompetitionId.count().max())
print('cantidad de teams:',df_cruce2.shape[0])

cantidad de lideres por team: 5
cantidad de teams: 686


In [31]:
print('cantidad de teams que no cruzaron',df_03.shape[0]-df_cruce2.shape[0])

cantidad de teams que no cruzaron 0


In [32]:
df_cruce2[df_cruce2['CompetitionId']==22111]

,CompetitionId,TagId,Name
545,22111,7100,biology
658,22111,16575,covid19
663,22111,4309,public health
664,22111,12005,biotechnology


## Listas de codigos de tags que cumplen la condición de:
### DS_toolkit: competencias con tags que se relacionan a Data Science - como un toolkit/herramientas de DS.
### DS_S: competencias con tags que se relacionan a Data Science para algoritmos supervizados (Clasificación/Regresión)
### DS_NS: competencias con tags que se relacionan a Data Science para algoritmos no supervizados (Cluster/PCA/FCA)

In [33]:
DS_toolkit = [6602,6603,6613,12101,12107,12125,12300,13102,13103,13104,13201,13202,13203,13204,13205,13206,13207,13208,13209,13210,13213,13215,13302,13304,13305,13306,13308,13309,13310,13311,13313,13315,13401,13402,13403,13404,13405,13406,13407,13408,13409,13410,13411,13412,13413,13414,13415,13416,13417,14101,14102,14104,14105,14106,14201,14202,14203,14211,14221,15002,16002,16008,16070,16071,16072,16074,16586,16587,16588,16589,16590,16591,16592,16593,16595,16596,16598,16599,16600,16601,16602,16603,16604,16605,16606,16607,16608,16609,16611,16612,16613,16614,16615,16616,16619,16620,16621,16622,16623,16624,16625,16627,16628,16629,16630,16631,16632,16633,16634,16635,16636,16639,16640,16641,16643,16668,16669]
DS_S = [6602,6603,6613,12101,12107,12125,12300,13102,13103,13104,13201,13202,13203,13205,13207,13208,13209,13210,13213,13215,13302,13305,13306,13308,13309,13310,13313,13315,13401,13402,13403,13404,13405,13406,13407,13410,13411,13412,13414,13415,13417,14101,14104,14105,14106,14201,14202,14203,14211,14221,15002,16002,16008,16070,16071,16072,16074,16586,16587,16588,16589,16590,16591,16592,16593,16595,16596,16598,16599,16600,16601,16602,16603,16604,16605,16606,16607,16608,16609,16611,16612,16613,16614,16615,16616,16619,16620,16621,16622,16623,16624,16625,16627,16628,16629,16630,16631,16632,16633,16634,16635,16636,16639,16640,16641,16643,16668,16669]
DS_NS = [6602,6603,6613,12101,12107,12125,12300,13102,13103,13104,13201,13202,13203,13204,13205,13206,13208,13210,13215,13304,13306,13308,13309,13310,13311,13315,13401,13408,13409,13410,13413,13416,14101,14102,14104,14105,14106,14211,14221,15002,16002,16008,16070,16071,16072,16074,16587,16588,16589,16590,16591,16593,16595,16596,16598,16599,16600,16601,16602,16603,16604,16606,16607,16608,16609,16611,16612,16613,16614,16615,16619,16620,16621,16622,16623,16624,16625,16627,16628,16629,16630,16631,16632,16633,16634,16635,16639,16640,16641,16643,16668,16669]

In [34]:
print(len(DS_toolkit))
print(len(DS_S))
print(len(DS_NS))

117
108
92


In [35]:
df_cruce2['flg_DS'] = np.where(df_cruce2[["TagId"]].isin(DS_toolkit),1,0)
df_cruce2['flg_DS_Supervizado'] = np.where(df_cruce2[["TagId"]].isin(DS_S),1,0)
df_cruce2['flg_DS_NoSupervizado'] = np.where(df_cruce2[["TagId"]].isin(DS_NS),1,0)

In [36]:
df_competencias_agg = df_cruce2.groupby('CompetitionId').agg({'flg_DS':['max'],'flg_DS_Supervizado':['max'],'flg_DS_NoSupervizado':['max']}).reset_index(drop=False)
df_competencias_agg.columns = ['CompetitionId','Marca_DS','Marca_Supervizado','Marca_NoSupervizado']

In [37]:
df_competencias_agg.head()

,CompetitionId,Marca_DS,Marca_Supervizado,Marca_NoSupervizado
0,3486,1,0,1
1,3526,1,1,1
2,3706,0,0,0
3,3810,1,1,1
4,3816,1,1,0


# Cruce de tablas para obtener la competencia para todos los usuarios de un team y no solo del lider
##### Tambien se le agrega la marca que caracteristica de DS tiene cada competencia

In [38]:
df_CompetenciaUsers = pd.merge(df_01[["TeamId","UserId"]], df_cruce1[["TeamId","CompetitionId"]],  how='left', left_on=['TeamId'], right_on = ['TeamId'])
df_CompetenciaUsersAttrib = pd.merge(df_CompetenciaUsers, df_competencias_agg,  how='left', left_on=['CompetitionId'], right_on = ['CompetitionId'])

In [39]:
print(df_01.shape)
print(df_CompetenciaUsers.shape)
print(df_CompetenciaUsersAttrib.shape)

(5478304, 2)
(5478304, 3)
(5478304, 6)


In [40]:
df_CompetenciaUsersAttrib['Marca_DS'] = np.where(df_CompetenciaUsersAttrib[['Marca_DS']].isnull(),0,df_CompetenciaUsersAttrib[['Marca_DS']])
df_CompetenciaUsersAttrib['Marca_Supervizado'] = np.where(df_CompetenciaUsersAttrib[['Marca_Supervizado']].isnull(),0,df_CompetenciaUsersAttrib[['Marca_Supervizado']])
df_CompetenciaUsersAttrib['Marca_NoSupervizado'] = np.where(df_CompetenciaUsersAttrib[['Marca_NoSupervizado']].isnull(),0,df_CompetenciaUsersAttrib[['Marca_NoSupervizado']])
df_CompetenciaUsersAttrib.head()

,TeamId,UserId,CompetitionId,Marca_DS,Marca_Supervizado,Marca_NoSupervizado
0,518,635,2408.0,0.0,0.0,0.0
1,526,839,2408.0,0.0,0.0,0.0
2,527,816,2408.0,0.0,0.0,0.0
3,528,778,2408.0,0.0,0.0,0.0
4,529,747,2408.0,0.0,0.0,0.0


In [41]:
print(df_CompetenciaUsersAttrib.Marca_DS.count())
print(df_CompetenciaUsersAttrib.Marca_DS.sum())

print(df_CompetenciaUsersAttrib.Marca_Supervizado.count())
print(df_CompetenciaUsersAttrib.Marca_Supervizado.sum())

print(df_CompetenciaUsersAttrib.Marca_NoSupervizado.count())
print(df_CompetenciaUsersAttrib.Marca_NoSupervizado.sum())

5478304
3659134.0
5478304
2946121.0
5478304
3481055.0


In [42]:
df_CompetenciaUsersAttrib[df_CompetenciaUsersAttrib["TeamId"]==111041]

,TeamId,UserId,CompetitionId,Marca_DS,Marca_Supervizado,Marca_NoSupervizado
144739,111041,38477,3947.0,1.0,0.0,1.0
144740,111041,223722,3947.0,1.0,0.0,1.0


In [43]:
df_cruce2[df_cruce2["CompetitionId"]==3947.0]

,CompetitionId,TagId,Name,flg_DS,flg_DS_Supervizado,flg_DS_NoSupervizado
1,3947,14102,image data,1,0,1


In [44]:
df_competencias_agg[df_competencias_agg["CompetitionId"]==3947.0]

,CompetitionId,Marca_DS,Marca_Supervizado,Marca_NoSupervizado
7,3947,1,0,1


# Tabla agregada a nivel usuario para saber los atributos de sus competencias
#### Saber el total de competencias que estuvo, cuantas de estas tienen caracteristicas de DS, DS Supervizado y DS No Supervizado

In [46]:
df_CompetenciaUsersAttrib_agg = df_CompetenciaUsersAttrib.groupby('UserId').agg({'UserId':['count'],'Marca_DS':['sum'],'Marca_Supervizado':['sum'],'Marca_NoSupervizado':['sum']}).reset_index(drop=False)
df_CompetenciaUsersAttrib_agg.columns = ['UserId','CTDCOMP','CTDCOMP_DS','CTDCOMP_Supervizado','CTDCOMP_NoSupervizado']
df_CompetenciaUsersAttrib_agg.head()

,UserId,CTDCOMP,CTDCOMP_DS,CTDCOMP_Supervizado,CTDCOMP_NoSupervizado
0,1,1,1.0,0.0,1.0
1,62,1,0.0,0.0,0.0
2,368,101,63.0,44.0,62.0
3,381,64,2.0,1.0,2.0
4,387,1,0.0,0.0,0.0


In [47]:
df_CompetenciaUsersAttrib_agg.shape

(1971901, 5)

In [49]:
df_CompetenciaUsersAttrib_agg.describe()

,UserId,CTDCOMP,CTDCOMP_DS,CTDCOMP_Supervizado,CTDCOMP_NoSupervizado
count,1.971901e+06,1.971901e+06,1.971901e+06,1.971901e+06,1.971901e+06
mean,3.916752e+06,2.778184e+00,1.855638e+00,1.494051e+00,1.765329e+00
std,2.809017e+06,5.451353e+00,4.059224e+00,3.240114e+00,3.895552e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.556049e+06,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.354141e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,5.905723e+06,3.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
max,1.094694e+07,1.695000e+03,2.800000e+02,2.160000e+02,2.680000e+02


In [48]:
df_CompetenciaUsersAttrib_agg.to_csv(r"../data/processed//UserAttributes.csv",index=False)